In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from server.database import engine, create_all, drop_all
from sqlalchemy import insert, select
from server.database.tables import user_table, login_history_table

# drop_all()
# create_all()

In [4]:
from sqlalchemy import text

# https://docs.sqlalchemy.org/en/14/tutorial/dbapi_transactions.html

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2021-09-29 01:09:13,519 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:09:13,519 INFO sqlalchemy.engine.Engine select 'hello world'
2021-09-29 01:09:13,520 INFO sqlalchemy.engine.Engine [generated in 0.00149s] {}
[('hello world',)]
2021-09-29 01:09:13,520 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
with engine.connect() as conn:
    with engine.connect() as conn:
        conn.execute(insert(user_table),
        [
            {'name': 'Kittie'},
        ])
        conn.commit()

2021-10-17 01:04:47,036 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-17 01:04:47,037 INFO sqlalchemy.engine.Engine INSERT INTO user (name) VALUES (%(name)s)
2021-10-17 01:04:47,037 INFO sqlalchemy.engine.Engine [generated in 0.00085s] {'name': 'Kittie'}
2021-10-17 01:04:47,062 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
with engine.connect() as conn:
    with engine.connect() as conn:
        result = conn.execute(
            select(user_table)
            .where(user_table.c.name == 'Kittie')
        )
        user_id = dict(result.one())['user_id']

user_id

2021-10-17 01:07:17,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-17 01:07:17,598 INFO sqlalchemy.engine.Engine SELECT user.user_id, user.name, user.password, user.joined_date 
FROM user 
WHERE user.name = %(name_1)s
2021-10-17 01:07:17,598 INFO sqlalchemy.engine.Engine [cached since 81.18s ago] {'name_1': 'Kittie'}
2021-10-17 01:07:17,600 INFO sqlalchemy.engine.Engine ROLLBACK


1

In [8]:
import time

with engine.connect() as conn:
    for _ in range(5):
        result = conn.execute(insert(login_history_table),
        [
            {'user_id': 1, 'confidence': 60},
        ])
        conn.commit()
        time.sleep(1)


2021-09-29 01:11:35,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:11:35,432 INFO sqlalchemy.engine.Engine INSERT INTO login_history (user_id, confidence) VALUES (%(user_id)s, %(confidence)s)
2021-09-29 01:11:35,432 INFO sqlalchemy.engine.Engine [generated in 0.00166s] {'user_id': 1, 'confidence': 60}
2021-09-29 01:11:35,443 INFO sqlalchemy.engine.Engine COMMIT
2021-09-29 01:11:36,465 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:11:36,466 INFO sqlalchemy.engine.Engine INSERT INTO login_history (user_id, confidence) VALUES (%(user_id)s, %(confidence)s)
2021-09-29 01:11:36,466 INFO sqlalchemy.engine.Engine [cached since 1.036s ago] {'user_id': 1, 'confidence': 60}
2021-09-29 01:11:36,467 INFO sqlalchemy.engine.Engine COMMIT
2021-09-29 01:11:37,488 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:11:37,489 INFO sqlalchemy.engine.Engine INSERT INTO login_history (user_id, confidence) VALUES (%(user_id)s, %(confidence)s)
2021-09-29 01:11:37,489 

In [8]:
from sqlalchemy import func

with engine.connect() as conn:
    result = conn.execute(
        select(func.count('*'))
        .select_from(login_history_table)
        .where(login_history_table.c.logout_date == None)
    )
    print(result.all()[0][0])

2
